In [1]:
pip install wrds

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wrds
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import sklearn.tree as tree
from IPython.display import Image  
import pydotplus
pd.set_option('display.max_rows', 1000)



In [3]:
db = wrds.Connection()

Enter your WRDS username [tim]:cyiu
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [4]:
#daily stock price
stocks = db.raw_sql("select datadate, tic, prccq, xrdq, actq, atq from comp.fundq WHERE datadate >= '2000-1-1' order by tic")

In [5]:
stocks.shape

(962577, 6)

In [7]:
stocks['1Q_return'] = np.where(stocks['tic']==stocks['tic'].shift(-1), \
                                  (stocks.prccq.shift(-1)/stocks.prccq-1),'0')

In [8]:
stocks = stocks.dropna()

In [9]:
stocks.shape

(248746, 7)

In [ ]:
#stocks['price_change'] = np.where(stocks['tic']==stocks['tic'].shift(-1), \
                                  #(100. * stocks.prccq.shift(-1)/stocks.prccq-1).round(1).astype(str) + '%','0')

In [ ]:
stocks[stocks.tic=='SQ']

In [ ]:
#def selectTicker():
   # entry = input('Enter ticker: ')
    #ticker = entry.upper()
   # return stocks.query(f'tic == "{ticker}"')

#x = selectTicker()

#x.head()

In [10]:
stocks["R&D / Assets"] = stocks["xrdq"]/stocks["atq"]

In [11]:
stocks.isna().any()

datadate        False
tic             False
prccq           False
xrdq            False
actq            False
atq             False
1Q_return       False
R&D / Assets     True
dtype: bool

In [12]:
# I noticed that there is outlier and the r&d over sales is inf, so i have decided to drop it
drop_max = stocks[stocks['R&D / Assets'] == stocks['R&D / Assets'].max()].index
stocks.drop(drop_max, inplace=True)
drop_min = stocks[stocks['R&D / Assets'] == stocks['R&D / Assets'].min()].index
stocks.drop(drop_min, inplace=True)

In [13]:
median = stocks['R&D / Assets'].median()

In [14]:
median

0.017858260726053826

In [15]:
# portfolio1 is the companies that have R&D / Assets above the threshold
portfolio1 = stocks[stocks['R&D / Assets'] >= median]
#portfolio2 is the companies that have R&D / Assets below the threshold
portfolio2 = stocks[stocks['R&D / Assets'] <= median]


In [16]:
portfolio1.head()

,datadate,tic,prccq,xrdq,actq,atq,1Q_return,R&D / Assets
3100,2000-03-31,0139A,37.7500,34.179,756.384,1671.196,-0.17880794701986757,0.020452
3101,2000-06-30,0139A,31.0000,35.031,764.385,1663.766,0.38104838709677424,0.021055
3102,2000-09-30,0139A,42.8125,34.879,808.337,1741.879,0.10072992700729921,0.020024
3103,2000-12-31,0139A,47.1250,53.609,911.742,1845.185,0.04466843501326245,0.029053
3104,2001-03-31,0139A,49.2300,41.687,983.382,1913.160,-0.3904123501929717,0.021790


In [22]:
portfolio1['1Q_return'] = portfolio1['1Q_return'].astype(float)

<ipython-input-22-a5965a9f2b37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio1['1Q_return'] = portfolio1['1Q_return'].astype(float)


In [25]:
test = portfolio1.groupby('tic')['1Q_return'].mean().sort_values()

In [26]:
test = test.dropna()

In [27]:
test

tic
NWTT      -0.999800
ABQQ      -0.996600
CHIT      -0.993617
CDDT      -0.975000
CGUD      -0.953333
            ...    
MXBTF    106.232143
XKEM     127.154334
CDWD     143.022573
BVTK     264.887565
PTIX     406.055106
Name: 1Q_return, Length: 6173, dtype: float64

In [28]:
Q1 = np.quantile(test,0.25)
Q3 = np.quantile(test,0.75)


In [34]:
test2 = test[(test.values>= Q1) & (test.values <= Q3)].sort_values()

In [36]:
test2[test2 > 0]

tic
CCHE      0.000026
LINN      0.000072
DMRC.1    0.000173
GLYC      0.000190
AKAOQ     0.000236
            ...   
XMEX      0.155550
PACB      0.155666
EVSNF     0.155708
ZSPH      0.155790
PROXQ     0.155945
Name: 1Q_return, Length: 2434, dtype: float64

In [38]:
len(test2[test2 <0])

522

In [ ]:
portfolio2.shape

In [ ]:
portfolio1['price_change'] = portfolio1.price_change.astype(float)
portfolio1.price_change.mean()

In [ ]:
portfolio2['price_change'] = portfolio2.price_change.astype(float)
portfolio2.price_change.mean()

In [ ]:
portfolio1_negative = portfolio1[portfolio1.price_change < 0]

In [ ]:
portfolio1_negative.price_change.mean()

In [ ]:
portfolio2_negative = portfolio2[portfolio2.price_change < 0]
portfolio2_negative.price_change.mean()

In [ ]:
portfolio1_positive = portfolio1[portfolio1.price_change > 0]
portfolio1_positive.price_change.mean()

In [ ]:
portfolio2_positive = portfolio2[portfolio2.price_change > 0]
portfolio2_positive.price_change.mean()

In [ ]:
portfolio1_positive.groupby('tic')['price_change'].count().sort_values()

In [ ]:
a = portfolio1.groupby('tic')['price_change'].mean()

In [ ]:
a

In [ ]:
Q1 = np.quantile(a.values,0.25)
Q3 = np.quantile(a.values,0.75)

In [ ]:
a[(a >= Q1) & (a <= Q3)]

In [ ]:
a = a.dropna()

In [ ]:
a.sort_values()

In [ ]:
portfolio1_positive = a[a.values > 0]

In [ ]:
portfolio1_positive

In [ ]:
Q1 = np.quantile(portfolio1_positive.values,0.25)
Q3 = np.quantile(portfolio1_positive.values,0.75)

In [ ]:
portfolio1_positive[(portfolio1_positive >= Q1) & (portfolio1_positive <= Q3)]

### 

In [ ]:
portfolio1_positive

In [ ]:
portfolio1

In [ ]:
Q1 = np.quantile(portfolio1.price_change,0.25)
Q3 = np.quantile(portfolio1.price_change,0.75)


In [ ]:
portfolio1.groupby('tic')['price_change'].mean()

In [ ]:
#portfolio2.sort_values(by = 'R&D / Assets', ascending = False).head(50)

In [ ]:
#Above_Five_Asset = stocks[stocks['atq'] >= 100]

In [ ]:
#Above_Five_Asset.sort_values(by = 'R&D / Assets', ascending = False).head(50)

In [ ]:
#Limited_Metric = Above_Five_Asset[Above_Five_Asset['R&D / Assets'] <= 0.02]

In [ ]:
#Limited_Metric.sort_values(by = 'R&D / Assets', ascending = False).head(10)

In [ ]:
#def selectTicker():
  #  df = None
   # entry = input('Enter ticker: ')
   # ticker = entry.upper()
   # df = Limited_Metric.query(f'tic == "{ticker}"')
   # if df.atq.min() < 200:
   #     print('Not a good investment.')
   #     selectTicker()
   # return df
    

#x = selectTicker()

#x

In [ ]:
#Limited_Metric.head()

In [ ]:
#Logitech = db.raw_sql("select datadate, tic, prccq, xrdq, actq, atq from comp.fundq where tic = 'LOGI'")

In [ ]:
#portfolio1['10to20%_return_in_1yr'] = ((portfolio1['future_1-yr_price_change'] >=0.1) & (portfolio1['future_1-yr_price_change'] <= 0.2 )) * 1
#portfolio1['20to30%_return_in_1yr'] = ((portfolio1['future_1-yr_price_change'] >=0.2) & (portfolio1['future_1-yr_price_change'] <= 0.3 )) * 1
#portfolio1['30to40%_return_in_1yr'] = ((portfolio1['future_1-yr_price_change'] >=0.3) & (portfolio1['future_1-yr_price_change'] <= 0.4 )) * 1
#portfolio1['40to50%_return_in_1yr'] = ((portfolio1['future_1-yr_price_change'] >=0.4) & (portfolio1['future_1-yr_price_change'] <= 0.5 )) * 1
#portfolio1['50%+_return_in_1yr'] = (portfolio1['future_1-yr_price_change'] >=0.5) * 1

In [ ]:
#portfolio2['10to20%_return_in_1yr'] = ((portfolio2['future_1-yr_price_change'] >=0.1) & (portfolio2['future_1-yr_price_change'] <= 0.2 )) * 1
#portfolio2['20to30%_return_in_1yr'] = ((portfolio2['future_1-yr_price_change'] >=0.2) & (portfolio2['future_1-yr_price_change'] <= 0.3 )) * 1
#portfolio2['30to40%_return_in_1yr'] = ((portfolio2['future_1-yr_price_change'] >=0.3) & (portfolio2['future_1-yr_price_change'] <= 0.4 )) * 1
#portfolio2['40to50%_return_in_1yr'] = ((portfolio2['future_1-yr_price_change'] >=0.4) & (portfolio2['future_1-yr_price_change'] <= 0.5 )) * 1
#portfolio2['50%+_return_in_1yr'] = (portfolio2['future_1-yr_price_change'] >=0.5) * 1

In [ ]:
#dt = tree.DecisionTreeClassifier(max_depth=4)

In [ ]:
#portfolio1.head()

In [ ]:
#dropxx = portfolio1[portfolio1.tic.isna()].index
#portfolio1.drop(dropxx,inplace=True)

In [ ]:
#portfolio1.isna().any()

In [ ]:
#drop99 = portfolio1[portfolio1['future_1-yr_price_change'].isna()].index
#portfolio1.drop(drop99,inplace=True)

In [ ]:
#drop98 = portfolio1[portfolio1['future_2-yr_price_change'].isna()].index
#portfolio1.drop(drop98,inplace=True)

In [ ]:
#drop97 = portfolio1[portfolio1['future_3-yr_price_change'].isna()].index
#portfolio1.drop(drop97,inplace=True)

In [ ]:
#portfolio1.isna().any()

In [ ]:
#X = portfolio1.drop({'datadate','20to30%_return_in_1yr','tic' },axis=1)

In [ ]:
#Y = portfolio1['20to30%_return_in_1yr']

In [ ]:
#dt.fit(X,Y)

In [ ]:
#dt_feature_names = list(X.columns)
#dt_target_names = [str(s) for s in Y.unique()]
#tree.export_graphviz(dt, out_file='tree.dot', 
    #feature_names=dt_feature_names, class_names=dt_target_names,
    #filled=True)  
#graph = pydotplus.graph_from_dot_file('tree.dot')
#Image(graph.create_png())

#If we want companies that have 20-30% return in 1 year and high R&D/Sales, 

In [ ]:
#portfolio1[(portfolio1['R&D / Sales'] <= 19103.25) & (portfolio1['txpd'] <= -16.334)]

In [ ]:
#dropa = portfolio2[portfolio2['future_1-yr_price_change'].isna()].index
#dropb = portfolio2[portfolio2['future_2-yr_price_change'].isna()].index
#dropc = portfolio2[portfolio2['future_3-yr_price_change'].isna()].index

In [ ]:
#portfolio2.drop(dropa,inplace=True)



In [ ]:
#dropb = portfolio2[portfolio2['future_2-yr_price_change'].isna()].index
#portfolio2.drop(dropb,inplace=True)


In [ ]:
#dropc = portfolio2[portfolio2['future_3-yr_price_change'].isna()].index
#portfolio2.drop(dropc,inplace=True)

In [ ]:
#portfolio2.dropna(how='any')

In [ ]:
#portfolio2.shape

In [ ]:
#X = portfolio2.drop({'datadate','20to30%_return_in_1yr','tic' },axis=1)

In [ ]:
#Y = portfolio2['20to30%_return_in_1yr']

In [ ]:
#dt.fit(X,Y)

In [ ]:
#dt_feature_names = list(X.columns)
#dt_target_names = [str(s) for s in Y.unique()]
#tree.export_graphviz(dt, out_file='tree.dot', 
    #feature_names=dt_feature_names, class_names=dt_target_names,
    #filled=True)  
#graph = pydotplus.graph_from_dot_file('tree.dot')
#Image(graph.create_png())

In [ ]:
#portfolio2[(portfolio2['future_1-yr_price_change'] >= 0.3) & (portfolio2['30to40%_return_in_1yr'] <=0.5)]

In [ ]:
# try to see which portfolio outperform the other
# median

In [ ]:
Agriculture = (stocks[(stocks.sich > 0) & (stocks.sich < 1000)])

In [ ]:
Agriculture.shape

In [ ]:
Mining = (stocks1[(stocks1.sich >= 1000) & (stocks1.sich < 1500)])

In [ ]:
Mining.shape

In [ ]:
Finance = (stocks1[(stocks1.sich >= 6000) & (stocks1.sich < 6800)])

In [ ]:
Finance.shape

In [ ]:
Services = (stocks1[(stocks1.sich >= 7000) & (stocks1.sich < 9000)])

In [ ]:
Services.shape

In [ ]:
Transportation = (stocks1[(stocks1.sich >= 4000) & (stocks1.sich < 5000)])

In [ ]:
Transportation.shape

In [ ]:
Manufacturing = (stocks1[(stocks1.sich >= 2000) & (stocks1.sich < 4000)])

In [ ]:
Manufacturing.shape

In [ ]:
stocks1["R&D / Sales"] = stocks1["xrd"]/stocks1["sale"]

In [ ]:
stocks1.head()

In [ ]:
Manufacturing.head()

In [ ]:
Manufacturing['R&D / Sales'].mean()

In [ ]:
Services['R&D / Sales'].mean()

In [ ]:
Manufacturing.sort_values(by = 'R&D / Sales', ascending = False).head(30)

In [ ]:
Manufacturing.groupby(by=["tic"]).mean()

In [ ]:
Manufacturing.sort_values(by = 'R&D / Sales', ascending = False)

In [ ]:
#threshold for sales
#scale
#problem is R&D/sales is highest in companies with very low sales
#Write up by June 1 -> most important
#Before 4pm
#R&D/Assets 
#Analysis of a more stable company
#Maybe different impact on stock return for both of these
#Market adjusted return
#What if they just invested in the market
#write it in a way that is easy for a non-business person